# 在交易策略中使用深度学习模型进行交易决策

在这个 Notebook 中我们将尝试训练一个简单的自定义序贯（Sequential）模型，用于判断是否应该在技术指标出现交易信号的时候进行交易。通常来讲，技术指标是基于数学公式所计算出的指标值。技术指标会随行情变化而变化。技术指标可以更直接地反映股市所处的状态，为交易提供指导。比如说，我们在之前的 BackTrader 策略示例中就以收盘价上穿均线作为开仓条件，收盘价下穿均线作为平仓条件。

然而，通过指标发出的买卖信号是一个随机过程，不可能非常准确。很多技术指标容易产生钝化，或发出一些错误的买卖信号。在本示例中，我们将以不同周期的技术指标为基础，通过机器学习模型做进一步的判断。您可以自行对模型进行调优并且观察这样做是否能够改善交易决策。

开始之前，先确保环境中有 Python 和 pip。建议您选择 Python 3 (Data Science) 内核完成实验。完成这个实验需要 TensorFlow 框架，需要安装最新版本。

In [ ]:
!pip install --upgrade pip -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install backtrader -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install tensorflow keras -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install matplotlib==3.1.3 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip uninstall -y sagemaker
!pip install sagemaker==1.72.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import sys

directory = '/root/sagemaker-backtrader-examples'
if directory not in sys.path:
    sys.path.append(directory)
    
output_bucket = 'athena-output-cache' # 之前创建的 Athena 输出桶名

然后还需要安装 TA-Lib，以生成训练所需的技术指标数值。我们提供了一个可以用于当前环境的 wheel 文件，可以通过以下命令用 pip 直接安装：

In [ ]:
!pip install talib-binary -i https://pypi.tuna.tsinghua.edu.cn/simple
import talib as ta

如果使用 wheel 文件安装后 import 报错，请解除以下代码的注释，下载源代码并重新编译：

In [ ]:
# !pip uninstall -y TA-Lib
# !git clone https://github.com/kynging/ta-lib
# !cd /root/ta-lib && apt install -y cmake gcc && ./configure --prefix=/usr && make && make install
# !pip install TA-Lib
# import talib as ta

## 准备工作

在这个部分我们将进行数据的准备、训练必要的模型并编写策略的回测脚本，用于稍后在 SageMaker Studio 中运行回测。

您可能注意到在本示例的路径下已经有生成的模型可供回测使用。如果时间有限的话，可以跳过此步骤直接运行回测。

### 模型训练

在这个演示中，我们将多获取自2010年至今的数据用于模型训练和预测。首先我们 import 必要的依赖包：

In [ ]:
import boto3
import datetime
import numpy as np
import pandas as pd
import talib as ta
import time

In [ ]:
import sys

directory = '/root/sagemaker-backtrader-examples'
if directory not in sys.path:
    sys.path.append(directory)
    
output_bucket = 'athena-output-cache' # 之前创建的 Athena 输出桶名

接下来定义一些必要的路径和名称“”

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role
import datetime
from sagemaker.tensorflow import TensorFlow
import json

role = get_execution_role()
session = sagemaker.Session()

aws_default_region = session.boto_session.region_name
aws_account_id = session.boto_session.client('sts').get_caller_identity()['Account']
image_repo_name = 'sagemaker-backtrader-examples'
image_tag = '3_strategy'
image = '{}.dkr.ecr.{}.amazonaws.com.cn/{}:{}'.format(aws_account_id, aws_default_region, image_repo_name, image_tag)
print('镜像：', image)

s3_bucket = 'stock-data-demo'
model_name = 'model-long-short-predict'
base_job_name = model_name
print('任务名：', base_job_name)

### 训练数据

以下的代码将从 Athena 中调取必要的数据：

In [ ]:
from data_util import get_query_result

database = 'stock-data-demo'
table = 'stock_day'
fields = 'ticker,tradedate,openprice,closeprice,highestprice,lowestprice,turnovervol,accumadjfactor,isopen'
end_date = '2020-4-13'
orderby = 'tradedate'
limit = '1006'
ticker = '600519'

query_string = f'''
SELECT DISTINCT {fields}
FROM "{database}"."{table}"
WHERE ticker='{ticker}'
AND tradedate<='{end_date}'
AND isopen=True
ORDER BY {orderby}
DESC
LIMIT {limit}
'''

df = get_query_result(query_string, output_bucket)

df['ticker'] = df['ticker'].apply(lambda x: str(x))
df['ticker'] = df['ticker'].apply(lambda x: '0'*(6-len(x)) + x)
df['openprice'] = df['openprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['closeprice'] = df['closeprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['highestprice'] = df['highestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['lowestprice'] = df['lowestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df.drop('isopen', 1, inplace=True)
df.drop('accumadjfactor', 1, inplace=True)
df.set_index('tradedate', inplace=True)
df.sort_index(0, inplace=True)
df.rename(columns={'openprice': 'open'}, inplace=True)
df.rename(columns={'closeprice': 'close'}, inplace=True)
df.rename(columns={'highestprice': 'high'}, inplace=True)
df.rename(columns={'lowestprice': 'low'}, inplace=True)
df.rename(columns={'turnovervol': 'volume'}, inplace=True)
df['openinterest'] = 0 # A股中一般并不考虑 interest 这一概念，先设为零

start_date = df.index[0]
end_date = df.index[-1]
print('Target stock:', ticker, start_date, '-', end_date)

df.head()

这里的代码将通过 TA-Lib 生成技术指标，并总结出在历史数据中技术指标的胜率统计：

In [ ]:
repeat_count = 15 # 
repeat_step = 1
look_back = repeat_count * repeat_step
forward_window = 5

profit_margin = 0.02 # 止盈条件为2%
stop_loss_margin = 0.01 # 止损条件为1%

closePrice = df["close"]

## 事先创建 DataFrame 的 header
header = ["tradedate", "close"]
for i in range(0, repeat_count):
    header.append("sma" + str((i+1) * repeat_step))
for i in range(0,repeat_count):
    header.append("roc" + str((i+1) * repeat_step))
header.append("long")
header.append("short")

print('header:', header)


data = []

## 通过 talib 计算出 SMA 和 ROC 值
# SMA - 即通常所说的算术平均值
inputs = {'close': np.array(closePrice)}
sma = [] # 包含15个不同周期的SMA时间序列
for i in range(0, repeat_count):
    sma.append(ta.SMA(np.array(closePrice), timeperiod=(i+1)*repeat_step + 1))
# ROC - Rate of change : ((price/prevPrice)-1)*100
roc = [] # 包含15个不同周期的ROC时间序列
for i in range(0, repeat_count):
    roc.append(ta.ROC(np.array(closePrice), timeperiod=(i+1)*repeat_step + 1))

## 筛选出触发止盈止损的交易日
long_count = 0 # 触发止盈次数
short_count = 0 # 触发止损次数
n_count = 0 #
n = 0
for idx in df.index:
    if n < len(df) - forward_window - 1:
        idx_0 = idx
        close_price = df.loc[idx, 'close']
        temp = []
        temp.append(idx)
        
        temp2 = []
        temp2.append(close_price)

        # sma
        for i in range(0, repeat_count):
            if np.isnan(sma[i][n]):
                temp2.append(close_price)
            else:
                temp2.append(sma[i][n])

        min_value = min(temp2)
        max_value = max(temp2)
        for i in temp2:
            if max_value == min_value:
                temp.append(0)
            else:
                temp.append((i - min_value) / (max_value - min_value))

        # roc
        for i in range(0, repeat_count):
            if np.isnan(roc[i][n]):
                temp.append(0)
            else:
                temp.append(roc[i][n])

        rClose = closePrice[(n+1):min(len(df)-1, n+1+forward_window)].values.tolist()
        min_value = min(rClose)
        max_value = max(rClose)
        
        # 止盈条件
        if max_value >= close_price * (1+profit_margin) and min_value >= close_price * (1-stop_loss_margin):
            long_count += 1
            temp.append(1)
        else:
            temp.append(0)

        # 止损条件
        if min_value <= close_price * (1-stop_loss_margin) and max_value <= close_price * (1+profit_margin):
            short_count += 1
            temp.append(1)
        else:
            temp.append(0)
        
        data.append(temp)
        n += 1

print("止盈：%s, 止损：%s" % (long_count,short_count))
df2 = pd.DataFrame(data, columns=header)
df2.set_index('tradedate', inplace=True)
print('tradedate:', df2.index[0], '-', df2.index[-1])

In [ ]:
prefix = '{}/input/data/training'.format(image_tag)
print()

清注意这些数据集共 1000 个交易日。您可能注意单纯按照技术指标交易产生的交易结果并不理想。

接下来我们对数据集进行简单的拆分，分为训练集和测试集两部分，并将数据上传到默认的 S3 路径：

In [ ]:
prefix = '{}/input/data/training'.format(image_tag)
key_prefix = '{}/{}'.format(image_repo_name, prefix)

df_train = df2.iloc[:int(0.6*len(df2))]
print('Training set:', df_train.index[0], '-', df_train.index[-1])
df_train.to_csv('{}/{}/data_train.csv'.format(directory, prefix))

df_test = df2.iloc[int(0.6*len(df2)):]
print('Testing set:', df_test.index[0], '-', df_test.index[-1])
df_test.to_csv('{}/{}/data_test.csv'.format(directory, prefix))

print('上传数据路径：', '{}/{}'.format(directory, prefix))
data_location = session.upload_data(directory+'/'+prefix, key_prefix=key_prefix)
print('S3数据路径：', data_location)
output_location = data_location[:data_location.find('input')] + 'output'
print('输出路径：', output_location)

### 模型训练

以下是我们实现准备的 Sequential 模型训练脚本：

In [ ]:
%%writefile {directory}/{image_tag}/model/train
#!/usr/bin/env python
from __future__ import print_function

import os
import sys
import traceback
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.layers import Dropout, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor

# Optional
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# These are the paths to where SageMaker mounts interesting things in your container.
prefix = '/opt/ml/'

input_path = os.path.join(prefix, 'input/data/training/data_train.csv')
test_path = os.path.join(prefix, 'input/data/training/data_test.csv')
output_path = os.path.join(prefix, 'output')
model_path = os.path.join(prefix, 'model')


# Process and prepare the data
def data_process(df, yLen, b):
    
    dataX = []
    dataY = []
    for idx, row in df.iterrows():
        row1 = []
        r = row[1:len(row)-yLen]
        for a in r:
            row1.append(a)
        x = np.array(row1, dtype=float)
        y = np.array(row[len(row)-yLen:], dtype=float)
        b = len(x)
        dataX.append(x)
        dataY.append(y)
        
    dataX = np.array(dataX)
    dataY = np.array(dataY)
    
    return dataX, dataY, b


def build_classifier(b, yLen):
    
    print("build_classifier:b=%s,yLen=%s" % (b, yLen))
    model = Sequential()
    model.add(Dense(b, input_dim=b, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(int(b/2), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(yLen,kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


def generate_model(dataX, dataY, b, yLen):
    
    model = build_classifier(b, yLen)
    model.fit(dataX, dataY, epochs=100, batch_size=1)
    scores = model.evaluate(dataX, dataY, verbose=0)
    print("Training Data %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    return model


def train():
    
    print('Starting the training.')
    
    yLen = 2
    b = 0
    
    try:
        
        df = pd.read_csv(input_path)
        dataX, dataY, b = data_process(df, yLen, b)
        print('b:', b, 'yLen:', yLen)
        model = generate_model(dataX, dataY, b, yLen)
        model.save(os.path.join(model_path, 'model.h5'))
        
        print('Training is complete. Model saved.')
        
        df = pd.read_csv(test_path)
        dataX, dataY, b = data_process(df, yLen, b)
        print('b:', b, 'yLen:', yLen)
        scores = model.evaluate(dataX, dataY, verbose=0)
        print("Test Data %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        
    except Exception as e:
        # Write out an error file. This will be returned as the failure
        # Reason in the DescribeTrainingJob result.
        trc = traceback.format_exc()
        with open(os.path.join(output_path, 'failure'), 'w') as s:
            s.write('Exception during training: ' + str(e) + '\n' + trc)
        # Printing this causes the exception to be in the training job logs
        print(
            'Exception during training: ' + str(e) + '\n' + trc,
            file=sys.stderr)
        # A non-zero exit code causes the training job to be marked as Failed.
        sys.exit(255)

        
if __name__ == '__main__':
    train()

    # A zero exit code causes the job to be marked a Succeeded.
    sys.exit(0)

以下的命令将通过 SageMaker SDK 远程调用一台实例进行模型训练：

In [ ]:
classifier = sagemaker.estimator.Estimator(
    image_name=image,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    output_path=output_location,
    sagemaker_session=session,
    base_job_name=base_job_name)

classifier.fit(data_location)

训练完成后的模型由 SageMaker 自动打包并上传至 S3。接下来我们将训练生成的名为 "model.h5" 的模型文件解压缩到 model/ 路径下：

In [ ]:
# 从 S3 中解压出模型文件

import boto3
import io
import tarfile

s3 = boto3.client('s3')
bucket = session.default_bucket()
s3_output_path = classifier.model_data.replace('s3://'+ bucket + '/', '')
print('s3路径：', s3_output_path)

bytestream = io.BytesIO(s3.get_object(Bucket=bucket, Key=s3_output_path)['Body'].read())
compressed_file = tarfile.open(fileobj=bytestream)
compressed_file.extractall(path=directory+'/3_strategy/')
print('解压缩路径：', directory+'/3_strategy/')

### 编写回测任务

接下来我们将定义运行回测任务所需的超参和代码。

#### 定义超参

这个算法中包含几个重要的参数：
  - long_threshold：当模型预测止盈概率超过此数值时做多 (0 到 1 之间的小数)。
  - short_threshold：当模型预测止损概率超过此数值时做空 (0 到 1 之间的小数)。因为 A 股不允许做空，设为 1 时策略就不会做空。
  - profit_target_pct：止盈条件（百分比）。请注意这个数值应该和训练中定义的 profit_margin 参数保持一致。
  - stop_target_pct：止损条件（百分比）。请注意这个数值应该和训练中定义的 stop_loss_margin 参数保持一致。
  - size：每次交易的股数。请注意股票的市场价格往往从几元到上千元不等，对不同的股票可能需要调整 size 参数或者调整回测初始的资金量。

In [ ]:
%%writefile {directory}/{image_tag}/input/config/hyperparameters.json
{ 
    "user" : "user",
    "long_threshold" : "0.5",
    "short_threshold" : "1",
    "profit_target_pct" : "2.00",
    "stop_target_pct" : "1.00",
    "size" : "60"
}

## 运行回测

### 调取回测数据

用于调取的数据集中通常不包含训练数据。您可以确认回测数据集的范围是和 testing 数据集基本一致的：

In [ ]:
from data_util import get_query_result

database = 'stock-data-demo'
table = 'stock_day'
fields = 'ticker,tradedate,openprice,closeprice,highestprice,lowestprice,turnovervol,accumadjfactor,isopen'
end_date = '2020-4-13'
orderby = 'tradedate'
limit = '406'
ticker = '600519'

query_string = f'''
SELECT DISTINCT {fields}
FROM "{database}"."{table}"
WHERE ticker='{ticker}'
AND tradedate<='{end_date}'
AND isopen=True
ORDER BY {orderby}
DESC
LIMIT {limit}
'''

df = get_query_result(query_string, output_bucket)

df['ticker'] = df['ticker'].apply(lambda x: str(x))
df['ticker'] = df['ticker'].apply(lambda x: '0'*(6-len(x)) + x)
df['openprice'] = df['openprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['closeprice'] = df['closeprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['highestprice'] = df['highestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['lowestprice'] = df['lowestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df.drop('isopen', 1, inplace=True)
df.drop('accumadjfactor', 1, inplace=True)
df.set_index('tradedate', inplace=True)
df.sort_index(0, inplace=True)
df.rename(columns={'openprice': 'open'}, inplace=True)
df.rename(columns={'closeprice': 'close'}, inplace=True)
df.rename(columns={'highestprice': 'high'}, inplace=True)
df.rename(columns={'lowestprice': 'low'}, inplace=True)
df.rename(columns={'turnovervol': 'volume'}, inplace=True)
df['openinterest'] = 0 # A股中一般并不考虑 interest 这一概念，先设为零

start_date = df.index[0]
end_date = df.index[-1]
print('Target stock:', ticker, start_date, '-', end_date)

df.head()
df.to_csv('{}/{}/backtest_data.csv'.format(directory, image_tag))

接下来的脚本包含了运行回测所需的策略代码。该策略将从示例的路径中加载之前训练好的模型。

In [ ]:
%%writefile {directory}/long_short_predict.py

import backtrader as bt
import json
import math
import numpy as np
import pandas as pd
import talib as ta
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import load_model


class MyStrategy(bt.Strategy):
    
    params=(('printlog', True),)

    def __init__(self):
        super(MyStrategy, self).__init__()
        
        directory = '/root/sagemaker-backtrader-examples'
        image_tag = '3_strategy'
        
        with open("{}/{}/input/config/hyperparameters.json".format(directory, image_tag)) as json_file:
            self.config = json.load(json_file)
        self.config["long_threshold"]=float(self.config["long_threshold"])
        self.config["short_threshold"]=float(self.config["short_threshold"])
        self.config["size"]=int(self.config["size"])
        self.config["profit_target_pct"]=float(self.config["profit_target_pct"])
        self.config["stop_target_pct"]=float(self.config["stop_target_pct"])

        self.order=None
        self.orderPlaced=False
                                
        self.model = load_model('{}/{}/model.h5'.format(directory, image_tag))
        
        # input / indicators
        self.repeat_count = 15
        self.repeat_step = 1
        
        self.profitTarget=self.config["profit_target_pct"]/100.0
        self.stopTarget=self.config["stop_target_pct"]/100.0
        self.size=self.config["size"]
    
        self.sma=[]
        self.roc=[]
        for i in range(0, self.repeat_count):
            self.sma.append(bt.talib.SMA(self.data, timeperiod=(i+1)*self.repeat_step + 1, plot=False))
            self.roc.append(bt.talib.ROC(self.data, timeperiod=(i+1)*self.repeat_step + 1, plot=False))
        
    def next(self):
        super(MyStrategy, self).next()
        
        idx_0 = self.datas[0].datetime.datetime(0)
        close_price = self.datas[0].close
        temp = []
        
        temp2 = []
        temp2.append(close_price)

        ## sma
        for i in range(0, self.repeat_count):
            if math.isnan(self.sma[i][0]):
                temp2.append(close_price)
            else:
                temp2.append(self.sma[i][0])

        min_value = min(temp2)
        max_value = max(temp2)
        for i in temp2:
            if max_value == min_value:
                temp.append(0)
            else:
                temp.append((i - min_value) / (max_value - min_value))

        ## roc
        for i in range(0, self.repeat_count):
            if math.isnan(self.roc[i][0]):
                temp.append(0)
            else:
                temp.append(self.roc[i][0])
        
        ## dataX
        dataX = np.array([np.array(temp)])

        ## dataY
        dataY = self.model.predict(dataX)
        
#         print(len(dataX[0]), len(dataY[0]))
        
        ## 开仓条件
        tLong = dataY[0][0]
        tShort = dataY[0][1]
        if not self.position:
            fLong = (tLong > self.config["long_threshold"]) 
            fShort = (tShort > self.config["short_threshold"])
            if fLong:
                self.order = self.buy(size=self.size)
                self.limitPrice = close_price + self.profitTarget*close_price
                self.stopPrice = close_price - self.stopTarget*close_price
            elif fShort:
                self.order = self.sell(size=self.size)                    
                self.limitPrice = close_price - self.profitTarget*close_price
                self.stopPrice = close_price + self.stopTarget*close_price

        ## 平仓逻辑
        if self.position:
            if self.position.size > 0:
                if close_price >= self.limitPrice or close_price <= self.stopPrice:
                    self.order = self.sell(size=self.size)
            elif self.position.size < 0:
                if close_price <= self.limitPrice or close_price >= self.stopPrice:
                    self.order = self.buy(size=self.size)
                    
    ## 日志记录
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()},{txt}')

    # 记录交易执行情况（可选，默认不输出结果）
    def notify_order(self, order):
        # 如果 order 为 submitted/accepted，返回空
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 如果 order 为 buy/sell executed，报告价格结果
        if order.status in [order.Completed]: 
            if order.isbuy():
                self.log(f'买入：\n价格：%.2f,\
                交易金额：-%.2f,\
                手续费：%.2f' % (order.executed.price, order.executed.value, order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'卖出:\n价格：%.2f,\
                交易金额：%.2f,\
                手续费：%.2f' % (order.executed.price, order.executed.price*self.size, order.executed.comm))
            self.bar_executed = len(self) 

        # 如果指令取消/交易失败, 报告结果
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('交易失败')
        self.order = None

    # 记录交易收益情况（可省略，默认不输出结果）
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        self.log(f'策略收益：\n毛收益 {trade.pnl:.2f}, 净收益 {trade.pnlcomm:.2f}')

    # 回测结束后输出结果（可省略，默认输出结果）
    def stop(self):
        self.log('期末总资金 %.2f' %
                 (self.broker.getvalue()), doprint=True)

In [ ]:
import long_short_predict

## 如果对策略进行了修改后想要重新加载策略，则运行以下的代码
import importlib
importlib.reload(long_short_predict)

## 运行回测任务

In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
import os.path
import sys

import backtrader as bt

if __name__ == '__main__':
    # 创建 Cerebro 对象
    cerebro = bt.Cerebro()

    # 创建 Data Feed
    df.index = pd.to_datetime(df.index)
    start = df.index[0]
    end = df.index[-1]
    print(start, '-', end)
    data = bt.feeds.PandasData(dataname=df, fromdate=start, todate=end)
    
    # 将 Data Feed 添加至 Cerebro
    cerebro.adddata(data)

    # 添加策略 Cerebro
    cerebro.addstrategy(long_short_predict.MyStrategy)
    
    # 设置初始资金
    cerebro.broker.setcash(100000.0)
    # 设置手续费为万二
    cerebro.broker.setcommission(commission=0.0002) 

    # 在开始时 print 初始账户价值
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # 运行回测流程
    cerebro.run()

    # 在结束时 print 最终账户价值
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
import matplotlib
%matplotlib inline

cerebro.plot(iplot=False)